<a href="https://colab.research.google.com/github/Kannee37/ThiGiacMay/blob/modul_detection/YOLO_cut_img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio
!pip install ultralytics
!pip install roboflow
!pip install opencv-python-headless
!pip install matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import ultralytics
print(ultralytics.__version__)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
8.3.100


## Tải data

In [3]:
from roboflow import Roboflow

rf = Roboflow(api_key="ccuIDfBPEMiseCibEMc8")
project = rf.workspace("cm-h8pey").project("biensoxe-mtw44")
version = project.version(1)
dataset = version.download("yolov11")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to BienSoXe-1 in yolov11:: 100%|██████████| 19880/19880 [00:05<00:00, 3610.77it/s]


In [5]:


# import zipfile
# import os

# # Specify the path to your zip file
# zip_file_path = "/content/runs.zip"

# # Extract the contents of the zip file to the current directory
# try:
#     with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#         zip_ref.extractall("/content")  # Specify the extraction directory
#     print(f"Successfully extracted {zip_file_path} to /content")
# except FileNotFoundError:
#     print(f"Error: {zip_file_path} not found.")
# except zipfile.BadZipFile:
#     print(f"Error: Invalid zip file: {zip_file_path}")
# except Exception as e:
#     print(f"An error occurred: {e}")



Successfully extracted /content/runs.zip to /content


In [15]:
import cv2
import os
from ultralytics import YOLO

# Tải mô hình YOLO đã huấn luyện
model = YOLO("/content/detect/train/weights/best.pt")  # Đường dẫn đến trọng số đã huấn luyện

# Đảm bảo thư mục chứa ảnh đầu vào và thư mục lưu ảnh đã cắt
output_dir = '/content/cropped_images/'  # Thư mục lưu ảnh đã cắt

# Tạo thư mục lưu ảnh nếu chưa có
os.makedirs(output_dir, exist_ok=True)


## Cắt ảnh biển số xe ra file riêng

In [16]:
import cv2
import os
from ultralytics import YOLO

# Định nghĩa hàm để cắt ảnh biển số xe từ mô hình YOLO
def crop_license_plates(input_dir, output_dir, model):
    # Tạo thư mục lưu ảnh nếu chưa có
    os.makedirs(output_dir, exist_ok=True)

    # Lặp qua tất cả các ảnh trong thư mục input_dir
    for filename in os.listdir(input_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):  # Kiểm tra định dạng ảnh
            img_path = os.path.join(input_dir, filename)

            # Đọc ảnh
            img = cv2.imread(img_path)

            # Dự đoán với mô hình YOLO
            results = model(img)

            # Kiểm tra số lượng bounding boxes được phát hiện
            detected_count = len(results[0].boxes)  # Sử dụng .boxes thay vì .xyxy
            if detected_count == 0:
                print(f"Không phát hiện biển số trong ảnh: {filename}")
            else:
                print(f"Phát hiện {detected_count} biển số trong ảnh: {filename}")

                # Lấy các bounding boxes và cắt ảnh biển số xe
                for i, result in enumerate(results[0].boxes):  # Kết quả của ảnh đầu tiên trong batch
                    # Lấy tọa độ bounding box và chuyển thành list
                    coords = result.xyxy.tolist()[0]  # Convert tensor to list and access the first element

                    # Gán giá trị của bounding box
                    xmin, ymin, xmax, ymax = map(int, coords)

                    # Cắt ảnh biển số xe
                    cropped_img = img[ymin:ymax, xmin:xmax]

                    # Tạo tên file ảnh cắt
                    output_filename = f"{os.path.splitext(filename)[0]}_cropped_{i}.jpg"
                    output_path = os.path.join(output_dir, output_filename)

                    # Lưu ảnh đã cắt
                    cv2.imwrite(output_path, cropped_img)
                    print(f"Đã lưu ảnh biển số xe tại: {output_path}")
    print("ĐÃ CẮT ẢNH XONG")

# Sử dụng hàm với dữ liệu của bạn
input_dir = '/content/BienSoXe-1/valid/images'  # Thư mục chứa ảnh test
output_dir = '/content/cropped_images'  # Thư mục lưu ảnh đã cắt
model = YOLO("/content/detect/train/weights/best.pt")  # Tải mô hình YOLO đã huấn luyện

# Gọi hàm để cắt ảnh
crop_license_plates(input_dir, output_dir, model)



0: 416x640 1 BSX, 129.2ms
Speed: 6.3ms preprocess, 129.2ms inference, 0.8ms postprocess per image at shape (1, 3, 416, 640)
Phát hiện 1 biển số trong ảnh: original_0157_02500_b_jpg.rf.5d73ef7469c6558e3c04fb1ef7e72a66.jpg
Đã lưu ảnh biển số xe tại: /content/cropped_images/original_0157_02500_b_jpg.rf.5d73ef7469c6558e3c04fb1ef7e72a66_cropped_0.jpg

0: 416x640 1 BSX, 126.2ms
Speed: 4.3ms preprocess, 126.2ms inference, 0.8ms postprocess per image at shape (1, 3, 416, 640)
Phát hiện 1 biển số trong ảnh: original_0249_02212_b_jpg.rf.79a5a3a167c623079272ec6fbe281e03.jpg
Đã lưu ảnh biển số xe tại: /content/cropped_images/original_0249_02212_b_jpg.rf.79a5a3a167c623079272ec6fbe281e03_cropped_0.jpg

0: 416x640 1 BSX, 142.6ms
Speed: 3.9ms preprocess, 142.6ms inference, 0.8ms postprocess per image at shape (1, 3, 416, 640)
Phát hiện 1 biển số trong ảnh: original_0259_04193_b_jpg.rf.d344c1b495838644bc02b42ed83b4996.jpg
Đã lưu ảnh biển số xe tại: /content/cropped_images/original_0259_04193_b_jpg.rf.

In [17]:
import os

# Kiểm tra xem thư mục đã lưu ảnh có bị rỗng hay không
cropped_images_path = '/content/cropped_images/'

# In ra danh sách các tệp trong thư mục
files_in_cropped_images = os.listdir(cropped_images_path)
print(f"Tất cả các tệp trong thư mục {cropped_images_path}: {files_in_cropped_images}")

# Kiểm tra nếu thư mục rỗng
if len(files_in_cropped_images) == 0:
    print("Thư mục cropped_images hiện tại rỗng.")
else:
    print(f"Thư mục cropped_images chứa {len(files_in_cropped_images)} tệp.")


Tất cả các tệp trong thư mục /content/cropped_images/: ['original_0209_02619_b_jpg.rf.b0729f8d195d79306b21847d3afca2f8_cropped_0.jpg', 'original_0256_04505_b_jpg.rf.143e7d4b251fbdd6946fbbcc2fc66497_cropped_0.jpg', 'original_0320_05789_b_jpg.rf.ce341f9f331fd0ef237feb45d11a3e74_cropped_0.jpg', 'original_0302_04345_b_jpg.rf.c65c82617c5a4f881efc832ba4eac663_cropped_0.jpg', 'original_0203_09520_b_jpg.rf.4083c1607e2b905704bfc16017932be6_cropped_0.jpg', 'original_0306_07265_b_jpg.rf.3b740abe34cd867558058264fc0018d6_cropped_0.jpg', 'original_0336_06983_b_jpg.rf.330b1d261744d2aaae77ed5ae6306631_cropped_0.jpg', 'original_0154_00518_b_jpg.rf.c71d256a7d9231ab6e63110e24cbec77_cropped_0.jpg', 'original_0345_00593_b_jpg.rf.da1fb7b5103225070c7d2bd4d7255d40_cropped_0.jpg', 'original_0249_05715_b_jpg.rf.49d810f0634ace2aee0cb6b2e6fa562f_cropped_0.jpg', 'original_0425_07085_b_jpg.rf.ab7aa7e957f20de3a42a5ea62bbf73c1_cropped_0.jpg', 'original_0317_04508_b_jpg.rf.0545f2ff53a612e49f723b5a87e214cc_cropped_0.jp

In [18]:
# tải folder cropped_images

import shutil

shutil.make_archive("cropped_images", 'zip', "/content/cropped_images")
from google.colab import files
files.download('cropped_images.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>